# Card Game with Augmented Reality

In [1]:
import cv2
import numpy as np
import time
import os
import import_ipynb
import card_identifiers
import math

# Constants

# This is used to seperate the background in the binary image
BKG_THRESH = 200 
# This is used when finding the corner contours
CORNER_TRESH = 180

# This is used to ignore other contourns that are too small or to big to be cards
CARD_MAX_AREA = 60000
CARD_MIN_AREA = 15000

# Dataset card Dimensions
CARD_WIDTH = 500
CARD_HEIGHT = 726

# Corner Identifier Dimensions
card_simple_rect = (30, 40, 130, 210)

importing Jupyter notebook from card_identifiers.ipynb


In [2]:
# Process binary image
def get_binary_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # blur for smoothing card contours
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    retval, thresh = cv2.threshold(blur, BKG_THRESH, 255, cv2.THRESH_BINARY)
    return thresh

In [3]:
# Get card contourns
def get_contourns(bynary_img):
    cnts, hier = cv2.findContours(bynary_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    index_sort = sorted(range(len(cnts)), key=lambda i: cv2.contourArea(cnts[i]), reverse=True)
    card_cnts = []

    if len(cnts) == 0:
        return []

    cnts_sort = []
    hier_sort = []
    cnt_is_card = np.zeros(len(cnts), dtype=int)

    for i in index_sort:
        cnts_sort.append(cnts[i])
        hier_sort.append(hier[0][i])

    for i in range(len(cnts_sort)):
        size = cv2.contourArea(cnts_sort[i])
        # Detect rectangular shape through perimeter
        peri = cv2.arcLength(cnts_sort[i], True)
        approx = cv2.approxPolyDP(cnts_sort[i], 0.01*peri, True)
        if ((size < CARD_MAX_AREA) and (size > CARD_MIN_AREA) and (hier_sort[i][3] == -1) and (len(approx) == 4)):
            card_cnts.append(cnts_sort[i])

    return card_cnts

In [4]:
# Get card's front view (homography)
def get_card_homography(contour, image):
    # Find perimeter of card and use it to approximate corner points
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.01*peri, True)
    pts = np.float32(approx)

    # bounding rect dimentions
    x, y, w, h = cv2.boundingRect(contour)

    # Apply homography
    warp = homography(image, pts, w, h)  

    return [warp, contour]

# Homography
def homography(image, pts, w, h):
    temp_rect = np.zeros((4, 2), dtype="float32")

    s = np.sum(pts, axis=2)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]

    diff = np.diff(pts, axis=-1)
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]

    if w < h:  # If card is vertically oriented
        temp_rect[0] = tl
        temp_rect[1] = tr
        temp_rect[2] = br
        temp_rect[3] = bl
    else:  # If card is horizontally oriented
        temp_rect[0] = bl
        temp_rect[1] = tl
        temp_rect[2] = tr
        temp_rect[3] = br

    # Create destination array, calculate perspective transform matrix and warp card image
    dst = np.array([[0, 0], [CARD_WIDTH-1, 0], [CARD_WIDTH-1,
                   CARD_HEIGHT-1], [0, CARD_HEIGHT-1]], np.float32)
    M = cv2.getPerspectiveTransform(temp_rect, dst)
    warp = cv2.warpPerspective(image, M, (CARD_WIDTH, CARD_HEIGHT))
    warp = cv2.cvtColor(warp, cv2.COLOR_BGR2GRAY)

    return warp


In [5]:
# Card identifiers are the corner images in the 'identifiers' folder
identifiers = card_identifiers.get_identifiers()
identifier_images = []
for identifier in identifiers:
    identifier_images.append(cv2.imread('identifiers/' + identifier.filename + '.png', cv2.IMREAD_UNCHANGED))

# Compares detected card's corner with dataset corner images
def get_card_id(corner):
    id = ''
    best_match = 0    
    for i, identifier in enumerate(identifiers):
        identifier_img = identifier_images[i]    
        resized_corner = cv2.resize(corner, (identifier_img.shape[1],identifier_img.shape[0]))
        difference = cv2.subtract(resized_corner,identifier_img)

        # resulting binary image of the difference between both corners
        subtracted = cv2.threshold(difference, 210, 255, cv2.THRESH_BINARY)[1]

        # Calculate difference through white pixels (result / original)
        corner_whites = float(np.sum(resized_corner == 255))
        subtracted_whites = float(np.sum(subtracted == 255))
        result = subtracted_whites / corner_whites
        if result > best_match:
            best_match = result
            id = identifier.filename
    return id

In [6]:
# Get and process card's corner
def get_card_corner(card, cnt):
    x1, y1, x2, y2 = card_simple_rect
    corner = card[y1:y2, x1:x2]
    retval, binary = cv2.threshold(corner, CORNER_TRESH, 255, cv2. THRESH_BINARY_INV)
    # Find contours in corner. Should have 2 or more contours. 1 for suit and 1 for number/name
    cnts, hier = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cropped_corner = None
    if(len(cnts)>1):
        # The two biggest contours are assumed to be the ones we need
        left_most_cnts = sorted(cnts, key=lambda ctr: cv2.boundingRect(ctr)[0])

        # To avoid having corner features merged with card's inner features,
        # crop the right side according to the comparative dimensions of the suit and number/name
        (x, y, w, h) = cv2.boundingRect(left_most_cnts[0])
        (x1, y1, w1, h1) = cv2.boundingRect(left_most_cnts[1])
        xx = min(x, x1)
        yy = min(y, y1)
        xx2 = max(x+w, x1+w1)
        yy2 = max(y+h, y1+h1)
        fix_thresh = max(x,x1) - xx
        xx2 = min(x+w, x1+w1) + fix_thresh

        cropped_corner = binary[yy:yy2, xx:xx2]
    return cropped_corner

In [7]:
# get Player position
def get_player(cnt,shape):
    height = shape[0]
    width = shape[1]
    diff = int((width-height)/2)

    # Player center positions
    left_most_point = [diff,height/2]
    right_most_point = [width-diff,height/2]
    top_most_point = [width/2,0]
    bottom_most_point = [width/2,height]

    # Card center position
    x, y, w, h = cv2.boundingRect(cnt)
    center = [int(x+w/2),int(y+h/2)]

    # Distances
    left_distance = math.dist(center,left_most_point)
    right_distance = math.dist(center,right_most_point)
    top_distance = math.dist(center,top_most_point)
    bottom_distance = math.dist(center,bottom_most_point)

    # player name
    index_min = np.argmin([left_distance,top_distance, right_distance, bottom_distance])
    player = ''
    if index_min == 0:
        player = "L"
    elif index_min == 1:
        player = "T"
    elif index_min == 2:
        player = "R"
    else:
        player = "B"

    return (int(x+w/2-50), int(y+h/2), player, index_min)

    


In [8]:
# Get most common card in player position
def get_common_card(cards):
    counter = 0
    card = cards[0]
     
    for i in cards:
        curr_frequency = cards.count(i)
        if(curr_frequency>= counter):
            counter = curr_frequency
            card = i
 
    return card

In [9]:
# Checks all 4 cards to see who gets the cards
def get_round_winner(first_card, cards):
    # Retunr -1 if there are not 4 cards in the table
    repetitions = set(cards)
    if len(repetitions) < 4:
        return -1
    
    # First card id
    first_card_id = first_card.split('_')
    name = first_card_id[0]
    suit = first_card_id[1]
    highest_card_index = 0
    highest_card_value = 0

    # check each card
    for i, card in enumerate(cards):
        card_id = card.split('_')
        # Check if is the right suit
        if card_id[1] != suit:
            continue
        name = card_id[0]
        value = 0
        if name == 'A':
            value = 14
        elif name == 'J':
            value = 11
        elif name == 'Q':
            value = 12
        elif name == 'K':
            value = 13
        else:
            value = int(name)
        if value > highest_card_value:
            highest_card_value = value
            highest_card_index = i
    return highest_card_index
    

In [10]:
# Time since it found any cards (to detect a new round)
search_time = 0

def get_search_time():
    return search_time

def set_search_time(value):
    search_time = value

In [11]:
# Round card recognition history for each player (4 players)
playing_cards = [[],[],[],[]]

def get_playing_cards():
    return playing_cards

def set_playing_cards(value):
    playing_cards = value

In [12]:
# Main function for card detection and identification
def get_cards(image, deltatime):
    # vars
    first_card = ''
    current_cards = ['','','','']
    playing_cards = get_playing_cards()
    search_time = get_search_time()

    # binary image
    binary = get_binary_image(image)

    # Test this in class room conditions
    #cv2.imshow('binary', binary)

    # Get card contours
    cnts = get_contourns(binary)
    cv2.drawContours(image, cnts, -1, (0, 255, 0), 2)

    # If there are no contours, do nothing
    card_ids = []
    common_cards = []
    if len(cnts) != 0:

        cards = []
        # For each contour...
        for i in range(len(cnts)):
            # Get front view of card
            card = get_card_homography(cnts[i], image)
            cards.append(card)

        if (len(cards) != 0):
            temp_cnts = []
            # Used to find the first card played
            most_concurrent_card = 0

            for i in range(len(cards)):
                temp_cnts.append(cards[i][1])
                # Get corner
                card_corner = get_card_corner(cards[i][0], cards[i][1])
                if card_corner is not None:
                    card_id = get_card_id(card_corner)
                    # If card was identified...
                    if card_id != '':
                        card_ids.append(card_id)
                        # Check card owner throught card's position
                        x,y, player, player_id = get_player(cards[i][1], image.shape)
                        playing_cards[player_id].append(card_id)
                        # Check player's card recognition history to avoid displaying possible outliers
                        common_card = get_common_card(playing_cards[player_id])
                        common_cards.append(common_card)
                        # Card needs to be displayed for 10 frames before having an id
                        if len(playing_cards[player_id])>10: 
                            current_cards[player_id] = common_card
                        cv2.putText(image,player + '-' + common_card,(x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(255,0,0),3,cv2.LINE_AA)
                        # Check if it was the first played card
                        if len(playing_cards[player_id]) > most_concurrent_card:
                            most_concurrent_card = len(playing_cards[player_id])
                            first_card = common_card
            # Draw all contours
            cv2.drawContours(image, temp_cnts, -1, (0, 255, 0), 2)
    # If no cards were found add time until 3 seconds have passed and clear round cards
    if len(card_ids) == 0:
        search_time = search_time + deltatime
        if search_time > 3:
            playing_cards = [[],[],[],[]]
            search_time = 0
    else:
        search_time = 0
    
    # set variables for next iteration
    set_playing_cards(playing_cards)
    set_search_time(search_time)

    # Check who gets the cards
    winner = get_round_winner(first_card, common_cards)
    return image, winner


In [13]:
'''
import imutils
img = cv2.imread('frames_test/frame_2.jpg')
img = imutils.resize(img, width=600, height=450)
get_cards(img,1)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''
